<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2024Project/blob/konstrukcja_mapy_pustyn/konstrukcja_mapy_pustyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import i instalacja potrzebnych bibliotek:

In [ ]:
!pip install datashader
!pip install holoviews hvplot colorcet
!pip install geoviews

In [ ]:
# import doinstalowanych pakietów
import datashader as ds
#import datashader.transfer_functions as tf
import colorcet as cc
import holoviews as hv
#from holoviews.operation.datashader import datashade
import geoviews as gv
import geoviews.tile_sources as gts
from holoviews import opts

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Klasyfikacja

In [ ]:
NASA = pd.read_csv('/content/drive/MyDrive/sampled_NASA_200k.csv')

In [ ]:
NASA['Date'] = NASA['Date'].astype(str)
NASA_lato = NASA[NASA['Date'].str[-2:].isin(["05", "06", "07", "08", "09", "10"])]

In [ ]:
selected_columns = ["lon", "lat", "Rainf", "Evap", "AvgSurfT", "Albedo", "SoilT_40_100cm", "GVEG"]
NASA_lato = NASA_lato[selected_columns].dropna()

In [ ]:
NASA_lato.head(5)

,lon,lat,Rainf,Evap,AvgSurfT,Albedo,SoilT_40_100cm,GVEG
0,-119.0625,42.4375,22.0672,7.629402,280.2746,40.87769,283.7812,0.030827
3,-74.1875,40.9375,125.9735,34.582930,288.0775,24.00672,284.2830,0.575912
4,-94.1875,30.0625,284.3076,132.679000,299.5116,24.61111,295.5570,0.615523
5,-117.0625,32.9375,1.4660,32.135890,295.4950,22.64583,290.7068,0.300523
7,-85.0625,43.4375,58.6264,83.969650,292.4481,24.78056,284.8408,0.737928


In [ ]:
CD = NASA_lato[(NASA_lato['lon'] >= -104) & (NASA_lato['lon'] <= -102) & (NASA_lato['lat'] >= 30) & (NASA_lato['lat'] <= 31)]
CP = NASA_lato[(NASA_lato['lon'] >= -110.5) & (NASA_lato['lon'] <= -108.5) & (NASA_lato['lat'] >= 39) & (NASA_lato['lat'] <= 40.5)]
GBD = NASA_lato[(NASA_lato['lon'] >= -116) & (NASA_lato['lon'] <= -114) & (NASA_lato['lat'] >= 40) & (NASA_lato['lat'] <= 41.5)]

In [ ]:
CD_i_niepustynia = NASA_lato[(NASA_lato['lon'] >= -106.5) & (NASA_lato['lon'] <= -104.5) & (NASA_lato['lat'] >= 32.5) & (NASA_lato['lat'] <= 33.5)]
CP_i_niepustynia = NASA_lato[(NASA_lato['lon'] >= -109) & (NASA_lato['lon'] <= -107) & (NASA_lato['lat'] >= 37.5) & (NASA_lato['lat'] <= 39)]
GBD_i_niepustynia = NASA_lato[(NASA_lato['lon'] >= -115) & (NASA_lato['lon'] <= -113) & (NASA_lato['lat'] >= 42.5) & (NASA_lato['lat'] <= 44)]

In [ ]:
niepustynia_przy_CD = NASA_lato[(NASA_lato['lon'] >= -109.5) & (NASA_lato['lon'] <= -107.5) & (NASA_lato['lat'] >= 33) & (NASA_lato['lat'] <= 34)]
niepustynia_przy_CP = NASA_lato[(NASA_lato['lon'] >= -107) & (NASA_lato['lon'] <= -105) & (NASA_lato['lat'] >= 39) & (NASA_lato['lat'] <= 40.5)]
niepustynia_przy_GBD = NASA_lato[(NASA_lato['lon'] >= -124) & (NASA_lato['lon'] <= -122) & (NASA_lato['lat'] >= 39.5) & (NASA_lato['lat'] <= 41)]

In [ ]:
GVEG_graniczne = round(
    pd.Series([
        CD['GVEG'].quantile(0.75),
        niepustynia_przy_CD['GVEG'].quantile(0.25),
        CP['GVEG'].quantile(0.75),
        niepustynia_przy_CP['GVEG'].quantile(0.25),
        GBD['GVEG'].quantile(0.75),
        niepustynia_przy_GBD['GVEG'].quantile(0.25),
    ]).mean(), 3
)

In [ ]:
Rainf_graniczne = round(
    pd.Series([
        CD_i_niepustynia['Rainf'].quantile(0.75),
        CD_i_niepustynia['Rainf'].quantile(0.25),
        CP_i_niepustynia['Rainf'].quantile(0.75),
        CP_i_niepustynia['Rainf'].quantile(0.25),
        GBD_i_niepustynia['Rainf'].quantile(0.75),
        GBD_i_niepustynia['Rainf'].quantile(0.25),
    ]).mean()
)

In [ ]:
Evap_graniczne = round(
    pd.Series([
        CD['Evap'].quantile(0.75),
        niepustynia_przy_CD['Evap'].quantile(0.25),
        CP['Evap'].quantile(0.75),
        niepustynia_przy_CP['Evap'].quantile(0.25),
        GBD['Evap'].quantile(0.75),
        niepustynia_przy_GBD['Evap'].quantile(0.25),
    ]).mean()
)

In [ ]:
AvgSurfT_graniczne = round(
    pd.Series([
        CD['AvgSurfT'].quantile(0.25),
        niepustynia_przy_CD['AvgSurfT'].quantile(0.75),
        CP['AvgSurfT'].quantile(0.25),
        niepustynia_przy_CP['AvgSurfT'].quantile(0.75),
        GBD['AvgSurfT'].quantile(0.25),
        niepustynia_przy_GBD['AvgSurfT'].quantile(0.75),
    ]).mean()
)

In [ ]:
Albedo_graniczne = round(
    pd.Series([
        CD['Albedo'].quantile(0.25),
        niepustynia_przy_CD['Albedo'].quantile(0.75),
        CP['Albedo'].quantile(0.25),
        niepustynia_przy_CP['Albedo'].quantile(0.75),
        GBD['Albedo'].quantile(0.25),
        niepustynia_przy_GBD['Albedo'].quantile(0.75),
    ]).mean(), 1
)

In [ ]:
SoilT_40_100cm_graniczne = round(
    pd.Series([
        CD['SoilT_40_100cm'].quantile(0.25),
        niepustynia_przy_CD['SoilT_40_100cm'].quantile(0.75),
        CP['SoilT_40_100cm'].quantile(0.25),
        niepustynia_przy_CP['SoilT_40_100cm'].quantile(0.75),
        GBD['SoilT_40_100cm'].quantile(0.25),
        niepustynia_przy_GBD['SoilT_40_100cm'].quantile(0.75),
    ]).mean()
)

In [ ]:
NASA = NASA[['lon', 'lat', 'Rainf', 'Evap', 'AvgSurfT', 'Albedo', 'SoilT_40_100cm', 'GVEG']].dropna()
NASA['klasyfikacja'] = np.nan

In [ ]:
def classify(row: pd.DataFrame):
    conditions = [
        row['Rainf'] <= Rainf_graniczne,
        row['Evap'] <= Evap_graniczne,
        row['GVEG'] <= GVEG_graniczne,
        row['AvgSurfT'] >= AvgSurfT_graniczne,
        row['Albedo'] >= Albedo_graniczne,
        row['SoilT_40_100cm'] >= SoilT_40_100cm_graniczne
    ]
    if np.nansum(conditions) >= 4:
        return "pustynia"
    else:
        return "nie-pustynia"

In [ ]:
NASA['klasyfikacja'] = NASA.apply(classify, axis=1)

In [ ]:
NASA.head(10)

,lon,lat,Rainf,Evap,AvgSurfT,Albedo,SoilT_40_100cm,GVEG,klasyfikacja
0,-119.0625,42.4375,22.067200,7.629402,280.274600,40.877690,283.781200,0.030827,pustynia
1,-91.6875,42.6875,56.069500,8.200099,271.731000,31.644440,279.603400,0.230389,nie-pustynia
2,-116.0625,50.3125,60.361220,0.511899,265.515700,65.425000,272.894700,0.203754,nie-pustynia
3,-74.1875,40.9375,125.973500,34.582930,288.077500,24.006720,284.283000,0.575912,nie-pustynia
4,-94.1875,30.0625,284.307600,132.679000,299.511600,24.611110,295.557000,0.615523,nie-pustynia
5,-117.0625,32.9375,1.466000,32.135890,295.495000,22.645830,290.706800,0.300523,pustynia
6,-112.0625,29.4375,0.000000,4.702000,294.737600,43.720830,292.038100,0.061700,pustynia
7,-85.0625,43.4375,58.626400,83.969650,292.448100,24.780560,284.840800,0.737928,nie-pustynia
8,-79.0625,36.1875,116.866585,103.839783,297.019257,24.379168,289.425293,0.779222,nie-pustynia
9,-102.4375,41.6875,26.514110,18.310310,277.460000,33.158600,282.275000,0.221730,pustynia


# Mapa pustyń

W zbiorze pojawiają się pary współrzędnych posiadające duplikaty, gdzie otrzymana klasyfikacja nie jest jednolita. W związku z tym, w celu wizualizacji, usuńmy duplikaty lokalizacji w taki sposób, żeby klasyfikacją była najczęściej występująca wartość wśród duplikatów.

In [ ]:
NASA_unique = NASA[['lon', 'lat', 'klasyfikacja']]

In [ ]:
NASA_unique

,lon,lat,klasyfikacja
0,-119.0625,42.4375,pustynia
1,-91.6875,42.6875,nie-pustynia
2,-116.0625,50.3125,nie-pustynia
3,-74.1875,40.9375,nie-pustynia
4,-94.1875,30.0625,nie-pustynia
...,...,...,...
199995,-102.6875,26.9375,pustynia
199996,-82.3125,52.0625,nie-pustynia
199997,-107.9375,37.1875,nie-pustynia
199998,-77.6875,52.6875,nie-pustynia


In [ ]:
grouped = NASA_unique.groupby(['lon', 'lat', 'klasyfikacja']).size().reset_index(name='count')
idx = grouped.groupby(['lon', 'lat'])['count'].transform(max) == grouped['count']
final_df = pd.DataFrame(grouped[idx][['lon', 'lat', 'klasyfikacja']])
# jeśli jakaś para współrzędnych ma równą ilość każdej z klasyfikacji to usuwamy tę lokalizację ze zbioru
final_df = final_df.groupby(['lon', 'lat']).filter(lambda x: len(x) == 1)

In [ ]:
NASA_for_map = final_df.copy()

Funkcja służąca stworzeniu mapy w formacie .html:

In [ ]:
def plot_map(df: pd.DataFrame, parameter_name: str, colormap: mpl.colors.LinearSegmentedColormap,
             point_size: int = 8, width: int = 900, height: int = 600, alpha: float = 1,
             bgcolor: str = 'white'):

    gdf = gv.Points(df, ['lon', 'lat'], [parameter_name]) # obiekt zawierający punkty
    tiles = gts.OSM # wybór mapy tła, w tym wypadku OpenStreetMap

    # łączenie mapy tła z punktami i ustawienie wybranych parametrów wizualizacji
    map_with_points = tiles * gdf.opts(
        color=parameter_name,
        cmap=colormap,
        size=point_size,
        width=width,
        height=height,
        colorbar=True,
        toolbar='above',
        tools=['hover', 'wheel_zoom', 'reset'],
        alpha=alpha # przezroczystość
    )

    map_with_points.opts(bgcolor=bgcolor)

    # zapis mapy do pliku .html
    output_filename = f'output_map_{parameter_name}.html'
    hv.save(map_with_points, output_filename)

Stworzenie mapy dla uzyskanych danych:

In [ ]:
plot_map(df=NASA_for_map, parameter_name='klasyfikacja', colormap=dict(zip(['pustynia', 'nie-pustynia'], ['yellow', 'green'])), point_size=3, alpha=0.7)

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
